![Introduction to Weaviate](./img/01-cover.png)

![Introduction to Weaviate](./img/02-about-weaviate.png)

### Agenda:

#### What you will see:

- Examples of AI-powered searches
- Create and build a vector database
- Search with a vector database
- Retrieval augmented generation (RAG)
- Scalability considerations

### You will learn:

- About vector, keyword & hybrid searches
    - When to use each one
- How to perform RAG
- How to build a scalable vector DB

## Search: An Introduction

Try searches using this (pre-populated) toy dataset. 

```json
animal_objs = [
    {"description": "brown dog"},
    {"description": "small domestic black cat"},
    {"description": "orange cheetah"},
    {"description": "black bear"},
    {"description": "large white seagull"},
    {"description": "yellow canary"},
]
```

In [ ]:
# Prep script: No need to show

import weaviate
from weaviate.classes.config import Configure, Property, DataType
from weaviate.classes.query import Filter, MetadataQuery
import os

# Recommended: save sensitive data as environment variables
cohere_key = os.getenv("COHERE_APIKEY")
headers = {
    "X-Cohere-Api-Key": cohere_key,
}

client = weaviate.connect_to_local(
    headers=headers
)

# Work with Weaviate

animals = client.collections.delete("Animals")

animals = client.collections.create(
    name="Animals",
    properties=[
        Property(name="description", data_type=DataType.TEXT),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="description",
            source_properties=["description"],
            api_endpoint="http://host.docker.internal:11434",  # If using Docker, use this to contact your local Ollama instance
            model="nomic-embed-text",  # The model to use, e.g. "nomic-embed-text"
        )
    ],
    generative_config=Configure.Generative.ollama(
        api_endpoint="http://host.docker.internal:11434",  # If using Docker, use this to contact your local Ollama instance
        model="gemma2:2b"
    ),
    # reranker_config=Configure.Reranker.cohere()
)

animal_objs = [
    {"description": "brown dog"},
    {"description": "small domestic black cat"},
    {"description": "orange cheetah"},
    {"description": "black bear"},
    {"description": "large white seagull"},
    {"description": "yellow canary"},
]

animals.data.insert_many(animal_objs)

### Traditional search

In [ ]:
query = "cat"

response = animals.query.bm25(query)

print(f"{len(response.objects)} results returned:")
for o in response.objects:
    print(o.properties)

But, traditional searches are not very robust. 

In [ ]:
query = "kitty"  # Try synonyms or even typos

response = animals.query.bm25(query)

print(f"{len(response.objects)} results returned:")
for o in response.objects:
    print(o.properties)

### Vector search

But vector search is based on similarity, allowing more forgiving, nuanced search:

In [ ]:
query = "cat"

response = animals.query.near_text(query)

print(f"{len(response.objects)} results returned:")
for o in response.objects:
    print(o.properties)

In [ ]:
query = "cat"  # Try synonyms or even typos

response = animals.query.near_text(query)

print(f"{len(response.objects)} results returned:")
for o in response.objects:
    print(o.properties)

Vector searches provide forgiving, nuanced, meaning-based similarity search. 

But - what is a vector?

## Introduction to Vectors

![Introduction to Vectors](./img/04-vectors-intro-01.png)

![Introduction to Vectors](./img/04-vectors-intro-02.png)

![Introduction to Vectors](./img/04-vectors-intro-03.png)

![Introduction to Vectors](./img/04-vectors-intro-04.png)

![Introduction to vectors](./img/04-vectors-intro-05.png)

## Why use vector search?

- Better search
    - Find contextually relevant info
    - Allow synonyms, different languages
    - More value from data
- Work together with generative AI models
    - Overcome hallucinations or lack of specific / prioprietary information

![Introduction to RAG](./img/06-rag-intro-01.png)

![Introduction to RAG](./img/06-rag-intro-02.png)

![Introduction to RAG](./img/06-rag-intro-03.png)

![Introduction to RAG](./img/06-rag-intro-04.png)

### Example RAG prompts:

- Summarise the corporate strategy of ACME Co for FY2024-25.
- What did the reviewers say about GadgetCo's SmartPhone12?
- How do you use Weaviate to perform RAG queries?

🤔 How well will these prompts work with *just* keyword searches?

![Introduction to RAG](./img/06-rag-intro-05.png)

## Build a database

## Preparation: Get the data

We'll use a dataset of movies from TMDB. 

Pre-processed version: "./data/movies.csv"


Load (or download) the data, and preview it

In [ ]:
import pandas as pd

# movie_df = pd.read_csv("./data/movies.csv")
movie_df = pd.read_csv("https://raw.githubusercontent.com/weaviate-tutorials/intro-workshop/main/data/movies.csv")
movie_df.head()

## Step 1: Connect to Weaviate

You can also use a hosted instance on Weaviate Cloud, or install Weaviate anywhere using the open-source distribution.

If you are using Cohere, or OpenAI, uncomment and update the relevant lines in the following cell with your actual keys. If yor using Ollama, you do not need to do anything.

In [ ]:
headers={
    # "X-Cohere-Api-Key": "<your_cohere_apikey>",  # Replace this with your actual key
    # "X-OpenAI-Api-Key": "<your_openai_apikey>",  # Replace this with your actual key
}

In [ ]:
import weaviate

# If you have got Weaviate running locally with Kubernetes or Docker:
client = weaviate.connect_to_local(
    port=80,  # Or 8080 for Docker instances
    headers=headers
)

# # If you are waiting for Docker to download, comment out the above, and uncomment this instead:
# client = weaviate.connect_to_embedded(
#     version="1.26.3",
#     headers=headers,
#     environment_variables={
#         "ENABLE_API_BASED_MODULES": "true"
#     }
# )

Retrieve Weaviate instance information to check our configuration.

In [ ]:
client.get_meta()

## Step 2: Add data to Weaviate

### Add collection definition

The equivalent of a SQL "table", is called a "collection" in Weaviate, like they are in NoSQL databases.

In case I created a demo collection - let's delete it.

In [ ]:
client.collections.delete("Movie")

And create a new collection definition here.
We'll set up a collection called "Movie" with:
- Two "named vectors" -> which will save different "meanings" of the data,
- A "generative" module -> which will allow us to use LLMs with our data, and
- Properties to save our movie data (which are like SQL columns).
    - Just the title, overview, year and popularity for now.

In [ ]:
from weaviate.classes.config import Configure, DataType, Property

client.collections.create(
    name="Movie",
    properties=[
        Property(
            name="title",
            data_type=DataType.TEXT,
        ),
        Property(
            name="overview",
            data_type=DataType.TEXT,
        ),
        Property(
            name="popularity",
            data_type=DataType.NUMBER,
        ),
        Property(
            name="year",
            data_type=DataType.INT,
        ),
    ],
    # ========================================
    # For those using Ollama:
    # ========================================
    vectorizer_config=[
        Configure.NamedVectors.text2vec_ollama(
            name="title",
            source_properties=["title"],
            api_endpoint="http://host.docker.internal:11434",  # If using Docker, use this to contact your local Ollama instance
            model="nomic-embed-text",  # The model to use, e.g. "snowflake-arctic-embed"
        ),
        Configure.NamedVectors.text2vec_ollama(
            name="all_text",
            source_properties=["title", "overview"],
            api_endpoint="http://host.docker.internal:11434",  # If using Docker, use this to contact your local Ollama instance
            model="nomic-embed-text",  # The model to use, e.g. "snowflake-arctic-embed"
        ),
    ],
    generative_config=Configure.Generative.ollama(
        api_endpoint="http://host.docker.internal:11434",
        model="gemma2:2b"
    ),
    # ========================================
    # END - Ollama setup
    # ========================================
    # # ========================================
    # # For those using Cohere:
    # # ========================================
    # vectorizer_config=[
    #     Configure.NamedVectors.text2vec_cohere(
    #         name="title",
    #         source_properties=["title"]
    #     ),
    #     Configure.NamedVectors.text2vec_cohere(
    #         name="all_text",
    #         source_properties=["title", "overview"]
    #     ),
    # ],
    # generative_config=Configure.Generative.cohere(),
    # # ========================================
    # # END - Cohere setup
    # # ========================================
    # # ========================================
    # # For those using OpenAI:
    # # ========================================
    # vectorizer_config=[
    #     Configure.NamedVectors.text2vec_openai(
    #         name="title",
    #         source_properties=["title"]
    #     ),
    #     Configure.NamedVectors.text2vec_openai(
    #         name="all_text",
    #         source_properties=["title", "overview"]
    #     ),
    # ],
    # generative_config=Configure.Generative.openai(),
    # # ========================================
    # # END - OpenAI setup
    # # ========================================
)

> Tip: You can get example collection definitions in our documentation:
> - https://weaviate.io/developers/weaviate/manage-data/collections

Was our collection created successfully? Let's take a look

In [ ]:
client.collections.exists("Movie")

### Add data

We'll add actual objects (SQL rows) to our data. 

First, let's build objects to add - and take a look at a couple.

In [ ]:
data_columns = ['title', 'overview', 'year', 'popularity']

df = movie_df[data_columns]

df.head()

> If it all looks fine - let's add objects:
> - https://weaviate.io/developers/weaviate/manage-data/import

In [ ]:
from tqdm import tqdm

movies = client.collections.get("Movie")

with movies.batch.fixed_size(200) as batch:
    for i, row in tqdm(df.iterrows()):
        obj_body = {
            c: row[c] for c in data_columns
        }
        batch.add_object(
            properties=obj_body
        )

#### Confirm data load

Do we have data? 

Let's get an object count

In [ ]:
print(len(movies))

Does the data look right?

Let's grab a few objects from Weaviate!

In [ ]:
response = movies.query.fetch_objects(limit=3)
for o in response.objects:
    print(o.properties)

Let's pause for a second - because we've done a lot!

#### What did we just do?

Here is a conceptual diagram

![img](https://github.com/weaviate-tutorials/intro-workshop/blob/main/images/object_import_process_full.png?raw=1)

## Step 3: Work with the data

Let's try a few more involved queries

### Filtering (similar to WHERE filter in SQL)

Let's find objects that meet a particular condition.

In [ ]:
from weaviate.classes.query import Filter

response = movies.query.fetch_objects(
    filters=Filter.by_property("year").greater_than(2015),
    limit=3
)

for o in response.objects:
    print(o.properties["title"])

But this does not rank the result in any meaningful way. 

For that, we need a keyword search (as opposed to a keyword *filter*).

### Keyword search

Unlike a keyword filter, a keyword search will search for, and rank results based on the frequency of the keyword.

In [ ]:
from weaviate.classes.query import MetadataQuery

response = movies.query.bm25(
    query="galaxy",
    limit=5,
    return_metadata=MetadataQuery(score=True, last_update_time=True)
)

for o in response.objects:
    print(o.metadata.score)
    print(o.metadata.last_update_time)
    print(o.properties)

### Semantic search

A semantic search, on the other hand, searches objects based on similarity

In [ ]:
import json

response = movies.query.near_text(
    query="galaxy",
    limit=3,
    target_vector="title",
)

for o in response.objects:
    print(json.dumps(o.properties, indent=2))

#### How does this work?

- Under the hood, this uses a vector search. It looks for objects which are the most similar to a text input.
- We can inspect the similarity along with the results.

In [ ]:
import json

response = movies.query.near_text(
    query="galaxy",
    limit=3,
    target_vector="title",
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.metadata)
    print(json.dumps(o.properties, indent=2))

This is where "vectors" come in. 

Each object in Weaviate includes a vector - like so:

In [ ]:
response = movies.query.near_text(
    query="galaxy",
    limit=3,
    target_vector="title",  # or "overview"
    include_vector=True,
    return_metadata=MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.metadata.distance)
    print(json.dumps(o.properties, indent=2))
    print(o.vector["title"])

These vector representations come from deep learning models to those that power LLMs. They capture meaning, and are called vector "embeddings".

### Generative search

A generative search transforms your data at retrieval time. 

In [ ]:
response = movies.generate.near_text(
    query="galaxy",
    limit=5,
    target_vector="title",
    single_prompt="Write a tweet promoting the movie with TITLE: {title} and OVERVIEW: {overview}.",
    grouped_task="What audience demographic might enjoy this group of movies?"
)

print(response.generated)
for o in response.objects:
    print(o.generated)
    print(json.dumps(o.properties, indent=2))

You can see here ⬆️ that each object has been transformed into a tweet by the LLM based on our prompt.

You can ask LLMs to perform all sorts of tasks

In [ ]:
response = movies.generate.near_text(
    query="galaxy",
    target_vector="title",
    limit=3,
    single_prompt="Summarise the following movie overview into a short French sentence: {overview}."
)

for o in response.objects:
    print(o.generated)
    print(json.dumps(o.properties, indent=2))

In [ ]:
client.close()